In [ ]:
# --- Importing the necessary libraries ---
import pandas as pd
import numpy as np

In [ ]:
# Data load (EDA's Copy)
df = pd.read_csv("../data/SPX.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")

first_valid_volume_date = df[df["Volume"] > 0].index[0]
df_model = df.loc[first_valid_volume_date:].copy()

df_model["Log_Return"] = np.log(df_model["Close"]/df_model["Close"].shift(1))
df_model["Volatility"] = df_model["Log_Return"].rolling(window=21).std()*np.sqrt(252) # Recuerda multiplicar por 100 si quieres la volatilidad en %
df_model["Momentum_126d"] = (df_model["Close"] - df_model["Close"].shift(126))/df_model["Close"].shift(126)
df_model.dropna(inplace=True)

print(df_model.head())

                 Open       High        Low      Close  Adj Close   Volume  \
Date                                                                         
1950-07-05  17.809999  17.809999  17.809999  17.809999  17.809999  1400000   
1950-07-06  17.910000  17.910000  17.910000  17.910000  17.910000  1570000   
1950-07-07  17.670000  17.670000  17.670000  17.670000  17.670000  1870000   
1950-07-10  17.590000  17.590000  17.590000  17.590000  17.590000  1960000   
1950-07-11  17.320000  17.320000  17.320000  17.320000  17.320000  3250000   

            Log_Return  Volatility  Momentum_126d  
Date                                               
1950-07-05    0.009591    0.271196       0.069028  
1950-07-06    0.005599    0.265688       0.062908  
1950-07-07   -0.013491    0.267615       0.043709  
1950-07-10   -0.004538    0.262510       0.035925  
1950-07-11   -0.015469    0.262583       0.014052  
